<img src='https://upload.wikimedia.org/wikipedia/commons/thumb/6/6e/Allianz_logo.svg/2560px-Allianz_logo.svg.png'>

# outlook-and-commentary

In [1]:
import requests
from bs4 import BeautifulSoup

from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import json
import re
from pprint import pprint
from tqdm import tqdm
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
outlook_and_commentary_url = 'https://www.allianzgi.com/en/insights/outlook-and-commentary'

In [73]:
driver = webdriver.Chrome()
driver.get(outlook_and_commentary_url)
sleep(5)
driver.find_element(By.CSS_SELECTOR, '#onetrust-accept-btn-handler').click()

while True:

    try:
        show_more_button = driver.find_element(By.CSS_SELECTOR, 'body > main > div > div > div.l-grid__row.align-items-stretch.js-list-view-container > div.u-flex.justify-content-center.u-margin-top-2m.show-more-center > a')    
        driver.execute_script("arguments[0].click();", show_more_button)

    except: 
        break

# for i in range(40):
#     show_more_button = driver.find_element(By.CSS_SELECTOR, 'body > main > div > div > div.l-grid__row.align-items-stretch.js-list-view-container > div.u-flex.justify-content-center.u-margin-top-2m.show-more-center > a')    
#     driver.execute_script("arguments[0].click();", show_more_button)
    finally:
        links = [i.get_attribute('href') for i in driver.find_elements(By.CSS_SELECTOR, "[class='l-grid u-padding-top-xl u-padding-bottom-xl js-list-view c-list-view'] [class='c-heading__link']")]
        title_list = [i.text for i in driver.find_elements(By.CSS_SELECTOR,"[class='l-grid__row align-items-stretch js-list-view-container'] a[class='c-heading__link']")]
        date_list = [i.text for i in driver.find_elements(By.CSS_SELECTOR,"[class='l-grid u-padding-top-xl u-padding-bottom-xl js-list-view c-list-view'] [class='c-agi-tile__footer']")]
        abstract_list = [i.text for i in driver.find_elements(By.CSS_SELECTOR, "[class='l-grid u-padding-top-xl u-padding-bottom-xl js-list-view c-list-view'] [class='c-copy c-agi-tile__text']")]

In [79]:
def to_date_format(raw_date):
    
    format_1 = re.findall(r'\b\d+\sdays ago\b',raw_date)
    format_2 = re.findall(r'\d+/\d+/\d+',raw_date)
    
    if bool(format_1) is True:
        n = int(''.join(re.findall(r'\d+',''.join(format_1))))
        extract_date = datetime.today() - timedelta(days=n)
        date = f'{extract_date.day}/{extract_date.month}/{extract_date.year}'
        return date
    
    elif bool(format_2) is True:
        date = ''.join(format_2)
        return date
    
    else:
        date = 'N/A'
        return date

In [92]:
def extract_content(link):
    
    # Send requrst and create BeautifulSoup object
    response = requests.get(link)
    soup = BeautifulSoup(response.text)
    
     # Tag
    tag = soup.find_all(class_='c-copy u-margin-bottom-none u-text-center u-margin-bottom-2xs')
    if bool(tag) is True: 
        tag = tag[0].text.strip()
    else:
        tag = 'N/A'
    
    # Content
    raw_content = soup.find_all(class_ = 'l-container l-container--no-gutters')
    if bool(soup.find_all(class_ = 'l-container l-container--no-gutters')) is True: 
        raw_content = ' '.join([i.text for i in raw_content])
        content = re.sub('\s+',' ', raw_content)
    else:
        content = 'N/A' 
    
    return content, tag

In [98]:
list_of_dict = []
index = 1

for title, date, link, abstract in tqdm(zip(title_list, date_list, links, abstract_list)):
    d = {'index': index,
         'company': 'Allianz', 
         'topic': 'Insight', 
         'insight_topic': 'Outlook & commentary',
         'title': title,
         'date': to_date_format(date),
         'link': link,
         'tag': extract_content(link=link)[1],
         'abstract': abstract,
         'content': extract_content(link=link)[0]
        }
    
    list_of_dict.append(d)
    index += 1

157it [06:55,  2.65s/it]


In [99]:
list_of_dict

[{'index': 1,
  'company': 'Allianz',
  'topic': 'Insight',
  'insight_topic': 'Outlook & commentary',
  'title': 'How China can achieve its ambitious decarbonisation plan',
  'date': '22/8/2022',
  'link': 'https://www.allianzgi.com/en/home/insights/outlook-and-commentary/ambitious-decarbonisation-plan',
  'tag': 'Appreciating China',
  'abstract': 'China’s pledge to achieve net-zero by 2060 appears ambitious, but it is central to the country’s broader strategic interests and the pathway to realising its goal could provide new opportunities for investors.',
  'content': ' Appreciating China How China can achieve its ambitious decarbonisation plan Virginie Maisonneuve | Shannon Zheng 8/22/2022 Share on Facebook Share on LinkedIn Share on Twitter Share via email China’s pledge to achieve net-zero by 2060 appears ambitious, but it is central to the country’s broader strategic interests and the pathway to realising its goal could provide new opportunities for investors. Key takeaways Chin

In [101]:
with open('allianz_insght_commentary.json', 'w') as f:
    json.dump(list_of_dict, f, indent=4)

In [96]:
extract_content('https://www.allianzgi.com/en/home/insights/outlook-and-commentary/ambitious-decarbonisation-plan')[1]

'Appreciating China'

In [ ]:
[class='l-grid u-padding-top-xl u-padding-bottom-xl js-list-view c-list-view'] [class='c-copy c-agi-tile__text']

In [74]:
abstract_list

['China’s pledge to achieve net-zero by 2060 appears ambitious, but it is central to the country’s broader strategic interests and the pathway to realising its goal could provide new opportunities for investors.',
 'New technologies are helping companies reinvent disrupted supply chains and build more resilience into the system.',
 'Investors are witnessing unprecedented market conditions. Within a turbulent market environment characterised by a high inflationary pressure, a shift from global to “glocal” trade patterns and a fading era of cheap energy, how can an active and agile multi asset approach be best placed to deliver returns?',
 'Supply chain strains, geopolitical tensions and a scaling back of bank funding are spurring demand for trade finance. Institutional investors can benefit from the potential for attractive returns and the short duration of the asset class.',
 'Copper is one of the most widely extracted metals, with only iron and aluminium being produced in greater quan

In [38]:
s1 = 'by\nVirginie Maisonneuve\n,\nShannon Zheng\n| 5 days ago'
s2 = 'by\nNeil Dwane\n| 10/04/2018'

In [41]:
dd = ''.join(re.findall(r'\b\d+\sdays ago\b',s1))

In [45]:
int(''.join(re.findall(r'\d+',dd)))

5

In [46]:
import datetime

In [52]:
datetime.date.today()

datetime.date(2022, 8, 28)

# Helper Functions

In [61]:
def to_date_format(raw_date):
    
    format_1 = re.findall(r'\b\d+\sdays ago\b',raw_date)
    format_2 = re.findall(r'\d+/\d+/\d+',raw_date)
    
    if bool(format_1) is True:
        n = int(''.join(re.findall(r'\d+',''.join(format_1))))
        extract_date = datetime.today() - timedelta(days=n)
        date = f'{extract_date.day}/{extract_date.month}/{extract_date.year}'
        return date
    
    elif bool(format_2) is True:
        date = ''.join(format_2)
        return date
    
    else:
        date = 'N/A'
        return date

In [35]:
re.findall(r'\d+/\d+/\d+',s2)

['10/04/2018']

In [62]:
to_date_format('by\nVirginie Maisonneuve\n,\nShannon Zheng\n| 5 days ago')

'23/8/2022'

In [63]:
ll = []
for i in tqdm(date_list):
    ll.append(to_date_format(i))

100%|█████████████████████████████████████████████████████████████████████████████| 227/227 [00:00<00:00, 75624.07it/s]


In [64]:
ll

['22/8/2022',
 '16/08/2022',
 '08/08/2022',
 '29/07/2022',
 '27/07/2022',
 '15/07/2022',
 '12/07/2022',
 '04/07/2022',
 '29/06/2022',
 '24/06/2022',
 '22/06/2022',
 '20/06/2022',
 '08/06/2022',
 '07/06/2022',
 '01/06/2022',
 '30/05/2022',
 '14/04/2022',
 '21/03/2022',
 '17/03/2022',
 '16/03/2022',
 '11/03/2022',
 '03/03/2022',
 '02/03/2022',
 '25/05/2022',
 '12/05/2022',
 '12/05/2022',
 '10/05/2022',
 '22/04/2022',
 '19/04/2022',
 '14/04/2022',
 '14/04/2022',
 '21/03/2022',
 '17/03/2022',
 '16/03/2022',
 '11/03/2022',
 '03/03/2022',
 '02/03/2022',
 '25/05/2022',
 '12/05/2022',
 '12/05/2022',
 '10/05/2022',
 '22/04/2022',
 '19/04/2022',
 '14/04/2022',
 '25/02/2022',
 '24/02/2022',
 '09/02/2022',
 '08/02/2022',
 '25/01/2022',
 '04/01/2022',
 '06/12/2021',
 '14/04/2022',
 '21/03/2022',
 '17/03/2022',
 '16/03/2022',
 '11/03/2022',
 '03/03/2022',
 '02/03/2022',
 '25/02/2022',
 '24/02/2022',
 '09/02/2022',
 '08/02/2022',
 '25/01/2022',
 '04/01/2022',
 '06/12/2021',
 '14/04/2022',
 '21/03/202

In [53]:
from datetime import datetime, timedelta
    
d = datetime.today() - timedelta(days=5)

In [56]:
a = d.date()

In [57]:
a

datetime.date(2022, 8, 23)

In [59]:
response = requests.get('https://www.allianzgi.com/en/home/insights/outlook-and-commentary/inflation-cheat-sheet#9a6f1002-2b42-49ec-865c-e1b54d1e43a2_0')
soup = BeautifulSoup(response.text)
# raw_content = ' '.join([i.text for i in soup.find_all('div',class_='l-grid__row')[11:]])
# content = re.sub('\s+', ' ', raw_content).strip()
# content
[i.text for i in soup.find_all(class_='c-tabs__content u-hidden')]

['\n\n\n\n\n\n\n\n\n\n\n\nInflation forecasts have risen in much of the developed world1\n\n\nConsensus CPI forecasts\n\n\n\n\n\n\n\n\n\n\n\nUS inflation is particularly high2\n\n\n\n\n\n\nHeadline CPI was up 9.1% year-over-year in June 2022 —\xa0the biggest jump since 1981.\xa0\n\n\nCore CPI was up 5.9% in the same period.\n\n\n\n\n\n\n\n\n\n\n\nWhy is this happening?\n\n\n\nWhen inflation began to rise notably in late 2021, many experts believed it was only transitory. But high inflation has persisted, raising concerns that we are entering a prolonged inflationary period.\n\n\nUS Federal Reserve Chairman Jerome Powell is concerned that "an extended period of high inflation could push longer-term expectations uncomfortably higher".\n\n\nWe have identified multiple factors pushing up inflation in the short and long term. Click the icons above to learn more.\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n3 factors pushing inflation up over the short term\n\n\n\n\n\n\nShrinking global output gap\n\nEcon

In [52]:
raw_content = ' '.join([i.text for i in soup.find_all('div',class_='l-grid__row')])

In [53]:
content = re.sub('\s+', ' ', raw_content).strip()

In [54]:
content

'Career What we do Our history Our culture Inclusion & diversity Corporate responsibility Art on climate Sponsorships & Partnerships People and places Our experts Our leadership team Our offices View All Sponsorships and Partnerships Great things happen when we work together, especially with a common goal. Learn more Corporate responsibility As a sustainable investor, we set high standards for the companies in which we invest – and our clients expect us to uphold the same standards in our own business. Read more Sponsorships and Partnerships Great things happen when we work together, especially with a common goal. Learn more Corporate responsibility As a sustainable investor, we set high standards for the companies in which we invest – and our clients expect us to uphold the same standards in our own business. Read more Allianz Global Investors Academy Outlook & commentary The Investment Intelligence Podcast Investable Themes Appreciating China Navigating Rates View All Insights Growth

In [58]:
raw_content = ' '.join([i.text for i in soup.find(class_='l-grid u-margin-top-m u-margin-bottom-m').find_all(class_='l-grid__row')[3:]])

In [61]:
content = re.sub('\s+',' ', raw_content)

In [57]:
soup.find_all('')

[]

In [62]:
content

' New technologies are helping companies reinvent disrupted supply chains and build more resilience into the system. Key-takeaways Supply chains are still reeling from the impact of Covid-19, war in Ukraine and deglobalisation in some sectors Upheaval to trade may impact corporate earnings, but companies are mitigating risks by shaking up their supply chains and investing in technology Amid the disruption, investors can find opportunities in sectors such as robotics, data centres, and agricultural technology that underpin the supply chain revolution While the recent resumption of grain shipments from the port of Odesa offers some relief to the myriad countries reliant on Ukraine for food staples, the supply chain stresses that have beset the global economy since 2020 are lingering. Shortages of goods from toilet paper to semiconductor chips have been a headache for consumers and businesses alike around the world over the past two years or so, due to factors ranging from Covid-19 to deg

In [42]:
def allianz_insghits(link):
    response = requests.get(link)
    soup = BeautifulSoup(response.text)
    raw_content = ' '.join([i.text for i in soup.find_all(class_='l-grid u-margin-top-m u-margin-bottom-m')])
    content = re.sub('\s+',' ', raw_content)
    return content

In [43]:
allianz_insghits('https://www.allianzgi.com/en/home/insights/outlook-and-commentary/german-lead-independent-directors')

''

In [53]:
tag = soup.find(class_='c-copy u-margin-bottom-none u-text-center u-margin-bottom-2xs').text.strip()

NameError: name 'soup' is not defined

In [89]:
title = soup.find(class_='c-heading--article-page u-text-center u-margin-bottom-xs u-text-weight-light').text.strip()

In [90]:
title

'How China can achieve its ambitious decarbonisation plan'

In [97]:
raw_date = soup.find(class_='author-container').text.strip()
date = ''.join(re.findall(r'\d+/\d+/\d+',date))

In [98]:
date

'8/22/2022'

In [100]:
links

['https://www.allianzgi.com/en/home/insights/outlook-and-commentary/ambitious-decarbonisation-plan',
 'https://www.allianzgi.com/en/home/insights/outlook-and-commentary/sparks-innovation-revolution',
 'https://www.allianzgi.com/en/home/insights/outlook-and-commentary/benefits-of-multi-asset-strategies',
 'https://www.allianzgi.com/en/home/insights/outlook-and-commentary/trade-finance',
 'https://www.allianzgi.com/en/home/insights/outlook-and-commentary/mineral-commodities-primer-copper',
 'https://www.allianzgi.com/en/home/insights/outlook-and-commentary/inflation-cheat-sheet',
 'https://www.allianzgi.com/en/home/insights/outlook-and-commentary/china-poised-to-bounce-back',
 'https://www.allianzgi.com/en/home/insights/outlook-and-commentary/german-lead-independent-directors',
 'https://www.allianzgi.com/en/home/insights/outlook-and-commentary/mid-year-outlook-chinas-path-out']

In [71]:
def allianz_insghits(link):
    
    # Send requrst and create BeautifulSoup object
    response = requests.get(link)
    soup = BeautifulSoup(response.text)
    
    # Title
    title = soup.find_all('h1')
    if bool(title) is True: 
        title = title[0].text.strip()
    else:
        title = 'N/A'
    
    # Date
    raw_date = soup.find_all(class_='c-copy u-margin-bottom-none')
    if bool(raw_date) is True: 
        raw_date = raw_date[0].text.strip()
        date = ''.join(re.findall(r'\d+/\d+/\d+',raw_date))
    else:
        date = 'N/A'
    
    # Tag
    tag = soup.find_all(class_='c-copy u-margin-bottom-none u-text-center u-margin-bottom-2xs')
    if bool(tag) is True: 
        tag = tag[0].text.strip()
    else:
        tag = 'N/A'
    
    # Content
    raw_content = soup.find_all(class_ = 'l-container l-container--no-gutters')
    if bool(soup.find_all(class_ = 'l-container l-container--no-gutters')) is True: 
        raw_content = ' '.join([i.text for i in raw_content])
        content = re.sub('\s+',' ', raw_content)
    else:
        content = 'N/A'
    
    allianz_insghi_dict = {
        'title': title,
        'date': date,
        'tag': tag,
        'content': content
    }
    
    
    
    
    return allianz_insghi_dict

In [93]:
allianz_insghits('https://www.allianzgi.com/en/home/insights/outlook-and-commentary/sparks-innovation-revolution')

{'title': 'Supply chain shake-up sparks innovation revolution',
 'date': 'N/A',
 'tag': '',
 'content': ' Supply chain shake-up sparks innovation revolution Virginie Maisonneuve 8/16/2022 Share on Facebook Share on LinkedIn Share on Twitter Share via email New technologies are helping companies reinvent disrupted supply chains and build more resilience into the system. Key-takeaways Supply chains are still reeling from the impact of Covid-19, war in Ukraine and deglobalisation in some sectors Upheaval to trade may impact corporate earnings, but companies are mitigating risks by shaking up their supply chains and investing in technology Amid the disruption, investors can find opportunities in sectors such as robotics, data centres, and agricultural technology that underpin the supply chain revolution While the recent resumption of grain shipments from the port of Odesa offers some relief to the myriad countries reliant on Ukraine for food staples, the supply chain stresses that have bes

In [72]:
try:
    for i in range(10):
        print(i)
        if i == 7 :
            raise Exception
except:
    pass
finally:
    print(i)

0
1
2
3
4
5
6
7
7
